<a href="https://colab.research.google.com/github/narsym/cat-and-dogs-classification/blob/master/Cat_and_Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initializing kaggle

In [1]:
!pip install kaggle

uploading kaggle.json to download dataset from the kaggle

In [0]:
from google.colab import files
files.upload()# upload your kaggle.json file

preparation

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Downloading dataset

In [4]:
!kaggle datasets download -d tongpython/cat-and-dog

 95% 208M/218M [00:04<00:00, 47.7MB/s]
100% 218M/218M [00:04<00:00, 50.5MB/s]


Unzipping dataset

In [0]:
!unzip cat-and-dog.zip

Importing libraries

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers,models,applications,preprocessing,optimizers
import cv2

Loading training data and testing data

In [0]:
train_data = []
train_labels = []
test_data = []
test_labels = []
IMG_SIZE = 160
def get_images(path,no_of_images):
  cfiles = os.listdir(os.path.join(path,'cats'))
  dfiles = os.listdir(os.path.join(path,'dogs'))
  data = []
  labels = []
  for i in dfiles:
    imgpath = os.path.join(path,'dogs') 
    img = cv2.imread(os.path.join(imgpath,i))
    img = tf.image.resize(img,(IMG_SIZE,IMG_SIZE))
    data.append(img)
    labels.append(1)
    if len(data) == no_of_images:
      break
  for i in cfiles:
    imgpath = os.path.join(path,'cats') 
    img = cv2.imread(os.path.join(imgpath,i))
    img = tf.image.resize(img,(IMG_SIZE,IMG_SIZE))
    data.append(img)
    labels.append(0)
    if len(data) ==  2 * no_of_images:
      break
  return data, labels

train_data, train_labels = get_images('./training_set/training_set',1000)
test_data, test_labels = get_images('./test_set/test_set',500)


Checking length

In [8]:
len(train_data),len(train_labels),len(test_data),len(test_labels)

(2000, 2000, 1000, 1000)

Creating tensorflow datasets for train and test data

In [0]:
train_data = tf.data.Dataset.from_tensor_slices((train_data,train_labels))
test_data = tf.data.Dataset.from_tensor_slices((test_data,test_labels))

Formatting images

In [0]:
def format_example(image,label):
  image = tf.cast(image,dtype = tf.float32)
  image = (image / 255) - 1
  image = tf.image.resize(image,(IMG_SIZE,IMG_SIZE))
  return image, label
train_data = train_data.map(format_example)
test_data = test_data.map(format_example)

Creating batches

In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 2000
train_data = train_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_data = test_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

Initializing MobileNetV2 model

In [12]:
IMG_SHAPE = (IMG_SIZE,IMG_SIZE,3)
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,include_top = False,weights = 'imagenet')
base_model.trainable = False
base_model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
________________

Checking the shape of batches in train_data

In [13]:
for image_batch, label_batch in train_data.take(1):
  pass
print(image_batch.shape)

(32, 160, 160, 3)


Checking the shape of output from basemodel

In [14]:
print(base_model(image_batch).shape)

(32, 5, 5, 1280)


Checking the shape of output from global average layer

In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
print(global_average_layer(base_model(image_batch)).shape)

(32, 1280)


Checking the shape of output from the hidden layer

In [16]:
hidden_layer = tf.keras.layers.Dense(32,activation = 'relu')
print(hidden_layer(global_average_layer(base_model(image_batch))).shape)

(32, 32)


Adding prediction layer and checking it's shape

In [17]:
prediction_layer = tf.keras.layers.Dense(1)
print(prediction_layer(hidden_layer(global_average_layer(base_model(image_batch)))).shape)

(32, 1)


Initializing the model with sequential

In [0]:
model = tf.keras.Sequential([
     base_model,
     global_average_layer,
     hidden_layer,
     prediction_layer
])

fixing learning rate and optimizer and compiling the model

In [0]:
base_learning_rate = 0.0001
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = base_learning_rate),
              loss = 'binary_crossentropy', metrics = ['accuracy'])

summary of the model

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                40992     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,299,009
Trainable params: 41,025
Non-trainable params: 2,257,984
_________________________________________________________________


calcualting the steps per epochs and validation steps

In [21]:
num_train = 2000
num_test = 1000
initial_epochs = 20
steps_per_epochs = round(num_train) //  BATCH_SIZE
validation_steps = 4

loss0, accuracy0 = model.evaluate(test_data, steps = validation_steps)

4/4 [==============================] - 0s 39ms/step - loss: 6.4293 - accuracy: 0.5703


Training the model

In [22]:
history = model.fit(train_data,epochs = initial_epochs, validation_data = test_data)

Epoch 1/20
63/63 [==============================] - 6s 93ms/step - loss: 2.9240 - accuracy: 0.6960 - val_loss: 1.0282 - val_accuracy: 0.8380
Epoch 2/20
63/63 [==============================] - 5s 83ms/step - loss: 0.7429 - accuracy: 0.8920 - val_loss: 0.6164 - val_accuracy: 0.9140
Epoch 3/20
63/63 [==============================] - 5s 82ms/step - loss: 0.4608 - accuracy: 0.9320 - val_loss: 0.5195 - val_accuracy: 0.9360
Epoch 4/20
63/63 [==============================] - 5s 82ms/step - loss: 0.3558 - accuracy: 0.9510 - val_loss: 0.4285 - val_accuracy: 0.9460
Epoch 5/20
63/63 [==============================] - 5s 82ms/step - loss: 0.2958 - accuracy: 0.9605 - val_loss: 0.3922 - val_accuracy: 0.9520
Epoch 6/20
63/63 [==============================] - 5s 83ms/step - loss: 0.2608 - accuracy: 0.9695 - val_loss: 0.3807 - val_accuracy: 0.9500
Epoch 7/20
63/63 [==============================] - 5s 82ms/step - loss: 0.2466 - accuracy: 0.9750 - val_loss: 0.3798 - val_accuracy: 0.9530
Epoch 8/20
63

Evaluating the model on the test set

In [23]:
model.evaluate(test_data)

32/32 [==============================] - 2s 48ms/step - loss: 0.4010 - accuracy: 0.9570


[0.4010383188724518, 0.9570000171661377]

We get 95% accuracy on test set.